<h3> Wordcount in CSV

In [5]:
from __future__ import print_function
import sys
import findspark
findspark.init()
from operator import add
from pyspark.sql import SparkSession

if __name__ == "__main__":
    # if len(sys.argv) != 2:
    #     print("Usage: wordcount <file>", file=sys.stderr)
    #     sys.exit(-1)

    spark = SparkSession\
        .builder\
        .appName("PythonWordCount")\
        .getOrCreate()

    df = spark.read.csv("/home/hdoop/PyWordCount/input.csv", header=True, inferSchema=True)
    lines = df.rdd.map(lambda r: r[0])
    
    counts = lines.flatMap(lambda x: x.split(' ')) \
                .map(lambda x: (x, 1)) \
                .reduceByKey(add)
    
    output = counts.collect()

    for (word, count) in output:
        print("%s: %i" % (word, count))

    spark.stop()


23/07/14 15:11:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


hello: 1
this: 1
: 8
is: 1
shreyash: 1
more: 1
and: 1
i: 1
am: 1
a: 1
fresher: 1


In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split

# Create SparkSession
csv_spark = SparkSession.builder.appName("CSVReader").getOrCreate()

# File path
csv_file_path = "input.csv"

# Read CSV file
df = csv_spark.read.csv(csv_file_path, header=True,inferSchema=True)
df.show()

text_column = 'THIS_IS_THE_CODE'

df = df.select(text_column)

# Split the text into words
words_df = df.select(explode(split(df[text_column], " ")).alias("word"))

word_counts = words_df.groupBy("word").count()

# Show the word count results
word_counts.show()

# Stop SparkSession
csv_spark.stop()

+----------------+
|THIS_IS_THE_CODE|
+----------------+
|           hello|
|           this |
|             is |
|       shreyash |
|           more |
|            and |
|              i |
|             am |
|              a |
|         fresher|
+----------------+

+--------+-----+
|    word|count|
+--------+-----+
|shreyash|    1|
|    more|    1|
|   hello|    1|
|      is|    1|
| fresher|    1|
|       i|    1|
|     and|    1|
|       a|    1|
|        |    8|
|    this|    1|
|      am|    1|
+--------+-----+

